In [2]:
import improve_utils as iu

In [3]:
responses_train = iu.load_single_drug_response_data_v2(source = 'CCLE', split_file_name='CCLE_split_3_train.txt', y_col_name='IC50')

Response data: (7615, 14)
Unique cells:  411
Unique drugs:  24


/lustre/work/statgrads/vpiyush2/DeepCDR_TF/improve_utils.py:187: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(improve_globals.y_file_path, sep=sep)


In [5]:
# responses_train

In [3]:
# first basically 

In [4]:
## the list of datasets we need are
## mutation
## gene expression
### copy number
## Dna methylation
## drug features

In [5]:
import pickle

In [6]:
with open("csa_data//common_genes.pickle", "rb") as f:
    common_genes = pickle.load(f).tolist()

In [7]:
# common_genes

In [8]:
# df_ge

In [9]:
df_ge = iu.load_gene_expression_data(gene_system_identifier="Gene_Symbol").columns
df_cnv = iu.load_copy_number_data(gene_system_identifier="Gene_Symbol").columns

Gene expression data: (1007, 30805)
Copy number data: (1018, 25331)


In [10]:
# df_cnv.dtypes

In [11]:
# df_cnv = df_cnv.fill

In [12]:
common_ftp = set(df_ge).intersection(df_cnv)

In [13]:
common_genes = set(common_genes).intersection(common_ftp)

In [14]:
import numpy as np

In [15]:
common_genes = np.sort(list(common_genes)).tolist()

In [16]:
with open("csa_data//tf_common_genes.pickle", "wb") as f:
    pickle.dump(np.sort(common_genes), f)

In [17]:
len(common_genes)

683

In [18]:
df_ge = iu.load_gene_expression_data(gene_system_identifier="Gene_Symbol")[common_genes]
df_cnv = iu.load_copy_number_data(gene_system_identifier="Gene_Symbol")[common_genes]
df_mut = iu.load_mutation_count_data(gene_system_identifier="Gene_Symbol")
df_methy = iu.load_dna_methylation_data(gene_system_identifier="TSS")

Gene expression data: (1007, 30805)
Copy number data: (1018, 25331)
Mutation count data: (1024, 18739)
DNA methylation data: (824, 19606)


In [19]:
# df_methy.dtypes

In [20]:
df_methy = df_methy.replace('     NA', np.nan)

In [21]:
df_methy = df_methy.astype("float64")

In [22]:
df_methy = df_methy.fillna(df_methy.mean())

In [23]:
df_ge.shape, df_cnv.shape, df_mut.shape, df_methy.shape

((1007, 683), (1018, 683), (1024, 18739), (824, 19606))

In [24]:
np.mean(np.array(df_ge.columns) == np.array(df_cnv.columns))

1.0

In [25]:
len(common_genes)

683

In [26]:
# common_genes

In [27]:
df_ge.shape

(1007, 683)

In [28]:
# common_genes

In [29]:
from sklearn.preprocessing import StandardScaler

In [30]:
import tensorflow as tf

2023-07-30 02:31:45.775042: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [31]:
import numpy as np

In [32]:
def get_emb_models(dataset, norm = False):
    std = StandardScaler()
    unique_ids = dataset.index
    text_vec_layer = tf.keras.layers.TextVectorization(max_tokens = dataset.shape[0] + 2, 
                                                  standardize=None, split = None, 
                                                  output_mode = "int", 
                                                  vocabulary = unique_ids.tolist())
    # weights = dataset.drop(id_col, 1).values
    weights = dataset.values
    padding_zeros = np.zeros((2, weights.shape[1]))
    weights = np.vstack((padding_zeros, weights))
    if norm == True:
        std.fit(weights)
        weights = std.transform(weights)
    emb_layer = tf.keras.layers.Embedding(dataset.shape[0] + 2, 
                                     weights.shape[1], 
                                     weights = [weights], 
                                     trainable = False)
    input_layer = tf.keras.layers.Input(shape = (1,), dtype = tf.string)
    vec_out = text_vec_layer(input_layer)
    emb_out = emb_layer(vec_out)
    flat_out = tf.keras.layers.Flatten()(emb_out)
    emb_model = tf.keras.models.Model(input_layer, flat_out)
    return emb_model

In [33]:
# df_ge.head()

In [34]:
cancer_gen_expr_model = get_emb_models(df_ge, norm = True)
cancer_gen_mut_model = get_emb_models(df_mut, norm = True)
cancer_dna_methy_model = get_emb_models(df_methy, norm = True)
cancer_gen_cnv_model = get_emb_models(df_cnv, norm = True)

2023-07-30 02:31:48.415401: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 02:31:49.154104: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78910 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:31:00.0, compute capability: 8.0


In [35]:
cancer_gen_expr_model.save("Models//cancer_gen_expr_model")
cancer_gen_mut_model.save("Models//cancer_gen_mut_model")
cancer_dna_methy_model.save("Models//cancer_dna_methy_model")
cancer_gen_cnv_model.save("Models//cancer_gen_cnv_model")

INFO:tensorflow:Assets written to: Models//cancer_gen_expr_model/assets


INFO:tensorflow:Assets written to: Models//cancer_gen_expr_model/assets


INFO:tensorflow:Assets written to: Models//cancer_gen_mut_model/assets


INFO:tensorflow:Assets written to: Models//cancer_gen_mut_model/assets


INFO:tensorflow:Assets written to: Models//cancer_dna_methy_model/assets


INFO:tensorflow:Assets written to: Models//cancer_dna_methy_model/assets


INFO:tensorflow:Assets written to: Models//cancer_gen_cnv_model/assets


INFO:tensorflow:Assets written to: Models//cancer_gen_cnv_model/assets


In [36]:
all_smiles = iu.load_smiles_data()

SMILES data: (1565, 2)


In [37]:
all_smiles.shape

(1565, 2)

In [38]:
all_smiles.head()

,improve_chem_id,smiles
0,Drug_0,F
1,Drug_1,Cl[Pt]Cl.N.N
2,Drug_10,O=P(O)(O)C(O)(Cn1ccnc1)P(=O)(O)O
3,Drug_100,CCOc1ccn(-c2ccc(F)cc2)c(=O)c1C(=O)Nc1ccc(Oc2cc...
4,Drug_1000,NCCCCC(NC(=O)C(N)CCCN=C(N)N)C(=O)NC(CCCCN)C(=O...


In [39]:
import deepchem as dc
from rdkit import Chem

In [40]:
def NormalizeAdj(adj):
    adj = adj + np.eye(adj.shape[0])
    d = sp.diags(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0).toarray()
    a_norm = adj.dot(d).transpose().dot(d)
    return a_norm
def random_adjacency_matrix(n):   
    matrix = [[random.randint(0, 1) for i in range(n)] for j in range(n)]
    # No vertex connects to itself
    for i in range(n):
        matrix[i][i] = 0
    # If i is connected to j, j is connected to i
    for i in range(n):
        for j in range(n):
            matrix[j][i] = matrix[i][j]
    return matrix
def CalculateGraphFeat(feat_mat,adj_list, Max_atoms):
    assert feat_mat.shape[0] == len(adj_list)
    feat = np.zeros((Max_atoms,feat_mat.shape[-1]),dtype='float32')
    adj_mat = np.zeros((Max_atoms,Max_atoms),dtype='float32')
    if israndom:
        feat = np.random.rand(Max_atoms,feat_mat.shape[-1])
        adj_mat[feat_mat.shape[0]:,feat_mat.shape[0]:] = random_adjacency_matrix(Max_atoms-feat_mat.shape[0])        
    feat[:feat_mat.shape[0],:] = feat_mat
    for i in range(len(adj_list)):
        nodes = adj_list[i]
        for each in nodes:
            adj_mat[i,int(each)] = 1
    assert np.allclose(adj_mat,adj_mat.T)
    adj_ = adj_mat[:len(adj_list),:len(adj_list)]
    adj_2 = adj_mat[len(adj_list):,len(adj_list):]
    norm_adj_ = NormalizeAdj(adj_)
    norm_adj_2 = NormalizeAdj(adj_2)
    adj_mat[:len(adj_list),:len(adj_list)] = norm_adj_
    adj_mat[len(adj_list):,len(adj_list):] = norm_adj_2    
    return [feat,adj_mat]

In [41]:
import scipy.sparse as sp

In [42]:
atom_list = []
for i, smiles in enumerate(all_smiles["smiles"].values):
    # print(each)
    molecules=[]
    molecules.append(Chem.MolFromSmiles(smiles))
    featurizer = dc.feat.graph_features.ConvMolFeaturizer()
    mol_object = featurizer.featurize(molecules)
    features = mol_object[0].atom_features
    atom_list.append(features.shape[0])

[02:31:54] WARNING: not removing hydrogen atom without neighbors
[02:31:54] WARNING: not removing hydrogen atom without neighbors
[02:31:54] WARNING: not removing hydrogen atom without neighbors


In [43]:
Max_atoms = np.max(atom_list)

In [44]:
Max_atoms

223

In [45]:
# all_smiles

In [46]:
israndom = False

In [47]:
dict_features = {}
# dict_num_atoms = {}
dict_adj_mat = {}
for i, smiles in enumerate(all_smiles["smiles"].values):
    # print(each)
    molecules=[]
    molecules.append(Chem.MolFromSmiles(smiles))
    featurizer = dc.feat.graph_features.ConvMolFeaturizer()
    mol_object = featurizer.featurize(molecules)
    features = mol_object[0].atom_features
    drug_id_cur = all_smiles.iloc[i,:]["improve_chem_id"]
    adj_list = mol_object[0].canon_adj_list
    l = CalculateGraphFeat(features,adj_list, Max_atoms)
    # dict_features[str(drug_id_cur)] = features
    # degree_list = mol_object[0].deg_list
    # degree_list = degree_list.reshape(-1,1)
    # dict_num_atoms[str(drug_id_cur)] = degree_list
    # padded = np.zeros((100-degree_list, 75))
    # features = np.vstack((features, padded))
    dict_features[str(drug_id_cur)] = l[0]
    dict_adj_mat[str(drug_id_cur)] = l[1]

[02:31:59] WARNING: not removing hydrogen atom without neighbors
[02:31:59] WARNING: not removing hydrogen atom without neighbors
[02:31:59] WARNING: not removing hydrogen atom without neighbors


In [48]:
# dict_features

In [49]:
# dict_adj_mat

In [50]:
all_smiles["improve_chem_id"].values

array(['Drug_0', 'Drug_1', 'Drug_10', ..., 'Drug_997', 'Drug_998',
       'Drug_999'], dtype=object)

In [51]:
dict_features['Drug_10'].sum()

85.0

In [52]:
with open("csa_data//drug_features.pickle", "wb") as f:
    pickle.dump(dict_features, f)

In [53]:
with open("csa_data//norm_adj_mat.pickle", "wb") as f:
    pickle.dump(dict_adj_mat, f)

In [54]:
PPI_file = "csa_data/PPI/PPI_network.txt"

In [55]:
# with open(selected_info_common_cell_lines) as f:
#     common_cell_lines = [item.strip() for item in f.readlines()]
    
with open("csa_data//tf_common_genes.pickle", "rb") as f:
    common_genes = pickle.load(f)
    
idx_dic={}
for index, item in enumerate(common_genes):
    idx_dic[item] = index

ppi_adj_info = [[] for item in common_genes] 
for line in open(PPI_file).readlines():
    gene1,gene2 = line.split('\t')[0],line.split('\t')[1]
    if (gene1 in common_genes) & (gene2 in common_genes):
        if idx_dic[gene1]<=idx_dic[gene2]:
            ppi_adj_info[idx_dic[gene1]].append(idx_dic[gene2])
            ppi_adj_info[idx_dic[gene2]].append(idx_dic[gene1])

In [56]:
# ppi_adj_info.shape

In [57]:
# common_genes

In [58]:
np.array(df_ge.columns).reshape(-1,1).shape

(683, 1)

In [59]:
common_genes.shape

(683,)

In [60]:
np.mean(np.array(df_ge.columns).reshape(-1,1) == common_genes.reshape(-1,1))

1.0

In [61]:
def CelllineGraphAdjNorm(ppi_adj_info,common_genes = common_genes):
    # with open(selected_info_common_genes) as f:
    #     common_genes = [item.strip() for item in f.readlines()]
    nb_nodes = len(common_genes)
    adj_mat = np.zeros((nb_nodes,nb_nodes),dtype='float32')
    # print(adj_mat.shape)
    for i in range(len(ppi_adj_info)):
        # print(i)
        nodes = ppi_adj_info[i]
        for each in nodes:
            adj_mat[i,each] = 1
    assert np.allclose(adj_mat,adj_mat.T)
    norm_adj = NormalizeAdj(adj_mat)
    return norm_adj 

In [62]:
# def NormalizeAdj(adj):
#     adj = adj + np.eye(adj.shape[0])
#     d = sp.diags(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0).toarray()
#     a_norm = adj.dot(d).transpose().dot(d)
#     return a_norm

In [63]:
import scipy.sparse as sp

In [64]:
ppi_adj = CelllineGraphAdjNorm(ppi_adj_info,common_genes)

In [65]:
ppi_adj = np.expand_dims(ppi_adj,0)

In [66]:
ppi_adj.shape

(1, 683, 683)

In [67]:
with open("csa_data//ppi_adj_matrix.pickle", "wb") as f:
    pickle.dump(ppi_adj, f)

In [68]:
common_achs = list(set(df_ge.index).intersection(df_cnv.index))

In [69]:
len(common_achs)

1000

In [70]:
# common_achs

In [71]:
common_ach_gen_expr = cancer_gen_expr_model(np.array(common_achs)).numpy()
common_ach_gen_cnv = cancer_gen_cnv_model(np.array(common_achs)).numpy()

In [72]:
common_ach_gen_expr = np.expand_dims(common_ach_gen_expr,-1)
common_ach_gen_cnv = np.expand_dims(common_ach_gen_cnv,-1)

In [73]:
common_ach_gen_cnv.shape

(1000, 683, 1)

In [74]:
common_ach_gen_expr_cnv = np.concatenate((common_ach_gen_expr, common_ach_gen_cnv), -1)

In [75]:
common_ach_gen_expr_cnv.shape

(1000, 683, 2)

In [76]:
combo_gen_expr_cnv_norm = ppi_adj@common_ach_gen_expr_cnv

In [77]:
combo_gen_expr_cnv_norm.shape

(1000, 683, 2)

In [78]:
dict1 = {}

counter = 0
for i in common_achs:
    dict1[i] = combo_gen_expr_cnv_norm[counter,:,:]
    counter += 1

In [79]:
# dict1

In [80]:
with open("csa_data//combo_gen_expr_cnv_norm.pickle", "wb") as f:
    pickle.dump(dict1, f)

In [81]:
# df_ge.sahape

In [82]:
# df_ge_cnv = np.concatenate([np.expand_dims(df_ge, -1),
#                                                       np.expand_dims(omics_gen_expr_train, -1)], axis = -1)

In [83]:
# cancer_copy_number_model.layers[1].get_vocabulary()

In [84]:
## make the normalized adjacency matrix 
## and the features matrix 



In [85]:
## then make the gcn features for the 